In [2]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

SparkContext.setSystemProperty('spark.executor.memory', '8g')
SparkContext.setSystemProperty('spark.driver.memory', '45G')

sc = SparkContext.getOrCreate()
spark = SparkSession.builder.appName("Python Spark").getOrCreate()

In [7]:
data = spark.read.json("data/hiphop.json")

In [8]:
data.printSchema()

root
 |-- acousticness: double (nullable = true)
 |-- analysis_url: string (nullable = true)
 |-- danceability: double (nullable = true)
 |-- duration_ms: long (nullable = true)
 |-- energy: double (nullable = true)
 |-- id: string (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- key: long (nullable = true)
 |-- liveness: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: long (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- time_signature: long (nullable = true)
 |-- track_href: string (nullable = true)
 |-- type: string (nullable = true)
 |-- uri: string (nullable = true)
 |-- valence: double (nullable = true)



In [ ]:
data

In [9]:
data.select(data.columns[:7]).show(10)

+------------+--------------------+------------+-----------+------+--------------------+----------------+
|acousticness|        analysis_url|danceability|duration_ms|energy|                  id|instrumentalness|
+------------+--------------------+------------+-----------+------+--------------------+----------------+
|      0.0536|https://api.spoti...|        0.91|     165067| 0.585|4KDNRh9Oor80z3XIx...|             0.0|
|       0.024|https://api.spoti...|       0.798|     138587| 0.574|4eL3XeuGaEoVT8ttD...|             0.0|
|     0.00311|https://api.spoti...|       0.723|     123077| 0.516|52nTQQahs46s1iHFQ...|         1.26E-6|
|      0.0635|https://api.spoti...|       0.849|     242966| 0.424|2BcMwX1MPV6ZHP4tU...|             0.0|
|      0.0252|https://api.spoti...|         0.8|     183514| 0.541|25r6MZ4Gtd76l43qn...|             0.0|
|      0.0698|https://api.spoti...|       0.707|     220413| 0.442|0uqsdBw9KvD8jSWmf...|             0.0|
|     0.00575|https://api.spoti...|       0.71

## Concatenation des records Spotify

In [21]:
import os

df = None
for f in os.listdir("data"):
    tmp = spark.read.json(os.getcwd() + "/data/" + f)
    if len(tmp.columns) != 0:
        df = df.union(tmp) if df is not None else tmp

In [24]:
print((df.count(), len(df.columns)))

(63081, 18)


In [23]:
df.dropDuplicates()

DataFrame[acousticness: double, analysis_url: string, danceability: double, duration_ms: bigint, energy: double, id: string, instrumentalness: double, key: bigint, liveness: double, loudness: double, mode: bigint, speechiness: double, tempo: double, time_signature: bigint, track_href: string, type: string, uri: string, valence: double]

In [26]:
df.printSchema()

root
 |-- acousticness: double (nullable = true)
 |-- analysis_url: string (nullable = true)
 |-- danceability: double (nullable = true)
 |-- duration_ms: long (nullable = true)
 |-- energy: double (nullable = true)
 |-- id: string (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- key: long (nullable = true)
 |-- liveness: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: long (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- time_signature: long (nullable = true)
 |-- track_href: string (nullable = true)
 |-- type: string (nullable = true)
 |-- uri: string (nullable = true)
 |-- valence: double (nullable = true)



In [53]:
col2ban = ['track_href', 'analysis_url', 'type', 'uri']
data = df.select([col for col in df.columns if col not in col2ban])
# reorder
data = data.select(['id'] + [col for col in data.columns if col != 'id'])

In [54]:
data.printSchema()

root
 |-- id: string (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- danceability: double (nullable = true)
 |-- duration_ms: long (nullable = true)
 |-- energy: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- key: long (nullable = true)
 |-- liveness: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: long (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- time_signature: long (nullable = true)
 |-- valence: double (nullable = true)



In [56]:
data.select(data.columns[:8]).show()

+--------------------+------------+------------+-----------+-------+----------------+---+--------+
|                  id|acousticness|danceability|duration_ms| energy|instrumentalness|key|liveness|
+--------------------+------------+------------+-----------+-------+----------------+---+--------+
|1vgSaC0BPlL6LEm4X...|       0.932|       0.257|     266200| 0.0906|           0.985|  8|  0.0786|
|2Bc4llhjJBW77I552...|       0.911|       0.167|     464293|  0.071|           0.855|  6|   0.107|
|4TrCvd1PrqI14lZb7...|       0.955|      0.0629|     272787| 0.0718|           0.907|  8|  0.0906|
|1i8MaQec4fQXj1enX...|       0.528|       0.196|     208040| 0.0236|          0.0845|  5|   0.289|
|0DsKfyDPVSfJPWb3J...|       0.953|       0.161|     717720|  0.101|           0.967|  5|  0.0651|
|16fZFSlV2VIDcY77m...|       0.947|       0.308|     290893|  0.251|           0.924|  2|  0.0707|
|2RWxRXIiwbcKEaplX...|       0.759|      0.0931|     392293|  0.273|           0.836|  9|  0.0888|
|3Q6okpLoq